## 言語モデルにおける会話

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

In [3]:
chat = ChatOpenAI(
    model="gpt-4o",
)
#
result = chat(
    [
        HumanMessage(content="だし巻き卵を作るのに必要な食材を教えて")
    ]
)
#
print(result.content)

だし巻き卵を作るために必要な基本的な食材は以下の通りです：

1. 卵：4個（一般的なレシピの場合）
2. だし：大さじ2（だし汁、または市販のだしパウダーを水で溶かしたもの）
3. 砂糖：小さじ1（お好みで量を調整）
4. 醤油：小さじ1（お好みで量を調整）
5. 塩：少々（お好みで量を調整）
6. 油：適量（卵焼き器に塗るため）

これらの食材を用意すれば、だし巻き卵を作ることができます。お好みに応じて、みりんを加える場合もあります。

作り方は以下の通りです：

1. ボウルに卵を割り入れ、よく溶きます。
2. 溶き卵にだし、砂糖、醤油、塩を加えてよく混ぜます。
3. 卵焼き器（またはフライパン）に油を薄く塗り、熱します。
4. 卵液を薄く広げ、固まり始めたら端から巻いていきます。
5. 卵液を追加し、再度薄く広げ、固まり始めたら巻いていきます。これを何度か繰り返します。
6. 形を整えながら焼き上げます。

焼き上がったら、少し冷ましてから切り分けてお召し上がりください。


In [4]:
result = chat(
    [
        HumanMessage(content="だし巻き卵を作るのに必要な食材を教えて"),
        AIMessage(content="""だし巻き卵を作るために必要な食材は以下の通りです。
                                1. 卵
                                2. だし（出汁）
                                3. 砂糖
                                4. 塩
                                5. しょうゆ
                                6. 油
                                """
                 ),
        HumanMessage(content="前回の回答を英語に翻訳して")
        
    ]
)
#
print(result.content)

Sure, here is the translation:

To make "dashimaki tamago" (Japanese rolled omelette with dashi), you need the following ingredients:
1. Eggs
2. Dashi (Japanese soup stock)
3. Sugar
4. Salt
5. Soy sauce
6. Oil


## 必要に応じた返答

In [5]:
from langchain.memory import ConversationBufferMemory

In [13]:
#バッファーを初期化
memory = ConversationBufferMemory(
    return_messages=True,
)

#メモリーにメッセージを追加
memory.save_context(
    {
        "input":"こんにちは！"
    },
    {
        "output":"こんにちは！お元気ですか？栄養の相談があればお気軽に。"
    }
)

#メモリーにメッセージを追加
memory.save_context(
    {
        "input":"今日は胃がもたれてます。"
    },
    {
        "output":"それなら、居酒屋川崎のだし巻き卵はいかがでしょう？"
    }
)

#メモリー内容を確認
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='こんにちは！'), AIMessage(content='こんにちは！お元気ですか？栄養の相談があればお気軽に。'), HumanMessage(content='今日は胃がもたれてます。'), AIMessage(content='それなら、居酒屋川崎のだし巻き卵はいかがでしょう？')]}


In [ ]:
!pip install chainlit

In [7]:
from langchain_openai import ChatOpenAI

## 会話履歴を保存し呼び出せるConversationBufferMemory

In [4]:
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage

#モデルを設定
chat = ChatOpenAI(
    model="gpt-4o",
)

#ConversationBufferMemoryを初期化
memory = ConversationBufferMemory(
    return_messages=True,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮したうえで返答ができるチャットボットです。").send()

@cl.on_message
async def on_message(message: cl.Message):
    #会話履歴のメモリーをロード
    memory_message_result = memory.load_memory_variables({})
    #辞書型データから会話の部分だけを配列に格納
    messages = memory_message_result['history']
    #今、入力されたメッセージを追加
    messages.append(HumanMessage(content=message.content))
    #会話履歴をそのまま渡す
    result = chat(
        messages
    )
    #メモリーにユーザー入力とLLMのレスを辞書型で保存
    memory.save_context(
        {
            "input": message.content,
        },
        {
            "output":result.content,
        }
    )
    #LLMの返答を返す
    await cl.Message(content=result.content).send()

## 会話履歴を考慮したチャットをConversationChainを使って簡単に

In [ ]:
#
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
#
from langchain.chains import ConversationChain

#モデルを設定
chat = ChatOpenAI(
    model="gpt-4o",
)

#会話バッファーを初期化
memory = ConversationBufferMemory(
    return_messages=True,
)

#ConversationChainをインスタンス化
chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮したうえで返答ができるチャットボットです。").send()

@cl.on_message
async def on_message(message: cl.Message):
    result = chain(
        message.content
    )
    #LLMの応答を返す
    await cl.Message(content=result["response"]).send()

In [ ]:
#上のコードをGeminiで
import chainlit as cl
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
#
from langchain.chains import ConversationChain

#モデルを設定
chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest"
)

#会話バッファーを初期化
memory = ConversationBufferMemory(
    return_messages=True,
)

#ConversationChainをインスタンス化
chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮したうえで返答ができるチャットボットです。").send()

@cl.on_message
async def on_message(message: cl.Message):
    result = chain(
        message.content
    )
    #LLMの応答を返す
    await cl.Message(content=result["response"]).send()

## 履歴をデータベースに保存

1.以下のURLにアクセス
https://upstash.com/

2.Create Database
- Name 
- Type Regional
- Region Japan

3.Key

4.Pass


In [15]:
#redisのインストール
!pip install redis

## Redisで会話を永続化

In [1]:
#RedisChatMessageHistoryでDBに接続できず
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import RedisChatMessageHistory  #← RedisChatMessageHistoryを追加
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(
    model="gpt-4o"
)

#← RedisChatMessageHistoryを初期化
history = RedisChatMessageHistory(  
    session_id="chat_history",
    url=os.environ["REDIS_URL"]
)

# history = RedisChatMessageHistory(session_id="chat_history",
#                                   url=os.environ["UPSTASH_REDIS_REST_URL"],
#                                   key_prefix=os.environ["UPSTASH_REDIS_REST_TOKEN"])


memory = ConversationBufferMemory(
    return_messages=True,
    chat_memory=history,
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)


@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮した返答をできるチャットボットです。メッセージを入力してください。").send()

@cl.on_message
async def on_message(message: str):
    #
    result=chain(message.content)
    #
    await cl.Message(content=result["response"]).send()

2024-05-22 16:09:58 - Loaded .env file


C:\Users\hartm\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 複数の会話履歴を持てるチャットボット

In [ ]:
#RedisChatMessageHistoryへの保存がどうも上手くいかない
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, RedisChatMessageHistory
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

@cl.on_chat_start
async def on_chat_start():
    thread_id = None
    #スレッドIDが入力されるまで繰り返す
    while not thread_id:
        #AskUserMessageを使ってスレッドIDを入力
        res = await cl.AskUserMessage(content="私は会話の文脈を考慮した返答ができるチャットボットです。スレッドIDを入力してください。", 
                                      timeout=600).send()
        if res:
            thread_id = res['content']
    #新しくチャットが始まるたびに初期化するようにon_chat_startに移動
    history = RedisChatMessageHistory(  
        session_id=thread_id,
        url=os.environ.get("REDIS_URL"),
    )
    
    #新しくチャットが始まるたびに初期化するようにon_chat_startに移動
    memory = ConversationBufferMemory( 
        return_messages=True,
        chat_memory=history,
    )
    #新しくチャットが始まるたびに初期化するようにon_chat_startに移動
    chain = ConversationChain( 
        memory=memory,
        llm=chat,
    )
    
    #メモリの内容を取得
    memory_message_result = chain.memory.load_memory_variables({})

    messages = memory_message_result['history']

    for message in messages:
        #ユーザーからのメッセージかどうかを判定
        if isinstance(message, HumanMessage): 
            #ユーザーからのメッセージの場合はauthorUserを指定して送信
            await cl.Message( 
                author="User",
                content=f"{message.content}",
            ).send()
        else:
            #AIからのメッセージの場合はChatBotを指定して送信
            await cl.Message(
                author="ChatBot",
                content=f"{message.content}",
            ).send()
    #履歴をセッションに保存
    cl.user_session.set("chain", chain)

@cl.on_message
async def on_message(message: cl.Message):
    #セッションから履歴を取得
    chain = cl.user_session.get("chain")

    result = chain(message.content)

    await cl.Message(content=result["response"]).send()

## 非常に長い会話に対応

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

chat=ChatOpenAI()

result=chat([
    HumanMessage(content="茶碗蒸しの作り方教えて"),
    AIMessage(content="{ChatModelからの返答である茶碗蒸しの作り方}"),
    HumanMessage(content="だし巻き玉子の作り方教えて"),
    AIMessage(content="{ChatModelからの返答であるだし巻き卵の作り方}"),
    HumanMessage(content="チャーハンの作り方教えて"),
])

print(result.content)

### 会話履歴の保存数を指定する場合

In [7]:
import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

#3往復分のメッセージを記憶する
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=3
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮した返答ができるチャットボットです。メッセージを入力してください。").send()

@cl.on_message
async def on_message(message: cl.Message):
    #保存されているメッセージを取得する
    messages = chain.memory.load_memory_variables({})["history"]
    #保存されているメッセージの数を表示する
    print(f"保存されているメッセージの数: {len(messages)}")
    #保存されているメッセージを1つずつ取り出す
    for saved_message in messages: 
        #保存されているメッセージを表示する
        print(saved_message.content)

    result = chain(message.content)

    await cl.Message(content=result["response"]).send()

### 会話履歴を要約してトークン数を抑える

In [ ]:
import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory 
from langchain.memory import ConversationSummaryMemory 
from langchain.chains import ConversationChain 
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain 

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

#ConversationSummaryMemoryを使用するように変更
#会話履歴を要約し1つにまとめる
memory = ConversationSummaryMemory(  
    llm=chat,
    return_messages=True,
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私は会話の文脈を考慮した返答をできるチャットボットです。メッセージを入力してください。").send()

@cl.on_message
async def on_message(message: cl.Message):
    #保存されているメッセージを取得する
    messages = chain.memory.load_memory_variables({})["history"]
    #要約されて１つになるのを確認用として
    print(f"保存されているメッセージの数: {len(messages)}")
    #保存されているメッセージを1つずつ取り出す
    for saved_message in messages:
        #保存されているメッセージを表示する
        print(saved_message.content)

    result = chain(message.content)

    await cl.Message(content=result["response"]).send()